In [36]:
import pandas as pd
from tqdm import tqdm

#display all words in column
pd.set_option('max_colwidth', None)
pd.set_option('display.max_columns', None)
from crossref_commons.retrieval import get_publication_as_json
from crossref_commons.iteration import iterate_publications_as_json

from utils import remove_html_tags, detect_language, split_entries_in_column,extract_FKZ

In [2]:
BMBF = pd.read_csv("../data/cleanedBMBF.csv")
BMBF.head(2)

,FKZ,Ressort,Referat,PT,Arb.-Einh.,Zuwendungsempfänger,Gemeindekennziffer,Stadt/Gemeinde,Ort,Bundesland,Staat,Ausführende Stelle,Gemeindekennziffer.1,Stadt/Gemeinde.1,Ort.1,Bundesland.1,Staat.1,Thema,Leistungsplansystematik,Klartext Leistungsplansystematik,Laufzeit von,Laufzeit bis,Fördersumme in EUR,Förderprofil,Verbundprojekt,Förderart
0,03F0212C/7,BMBF,724,PT-J,MGS1,Universität Rostock,13003000.0,Rostock,Rostock,Mecklenburg-Vorpommern,Deutschland,Universität Rostock - Fachbereich Elektrotechnik und Informationstechnik,13003000.0,Rostock,Rostock,Mecklenburg-Vorpommern,Deutschland,"Verbundprojekt MESSIN: Entwicklung und Systemintegration der Komponenten Navigation, Automatische Steuerung, Kommunikation und Energieversorgung für den Meßdelphin MESSIN",FB4010,"Technik-/Infrastrukturentwicklung und -bereitstellung (Überwachungssysteme, Begleitforschung zum Monitoring, innovative Technik und Geräteträger)",01.01.1998,31.08.2000,"648.462,00",Forschung und Entwicklung zur Daseinsvorsorge,MESSIN,PDIR
1,D251800/0,BMBF,314,BIBB,A3.3,"Ministerium für Wirtschaft, Arbeit und Wohnungsbau Baden-Württemberg",8111000.0,Stuttgart,Stuttgart,Baden-Württemberg,Deutschland,KACO new energy GmbH,8121000.0,Heilbronn,Heilbronn,Baden-Württemberg,Deutschland,Prozeßorientierte Entwicklungsplanung und Qualifizierung im Betrieb,OB1050,Versuchs- und Modelleinrichtungen und -programme im Bereich der beruflichen Bildung,01.07.1997,30.06.2000,"96.634,00",Forschung und Entwicklung zur Daseinsvorsorge,NaN,PDIR


### At CROSSREF, each funder is assigned to a so-called funder number, which is unique and is used to identify the oganisation/ funder . 501100002347 is the funder number of BMBF . So we will retrieve all metadatas which are related to this funder. To do this, we make use of the python modul crossref-commons (https://api.crossref.org/swagger-ui/index.html)

In [6]:
def retrieve_metadata(funder='501100002347', max_results = 45000):
    """ function to retrieve metadatas from publication funded by BMBF """
    
    
    metadatas = []
    filters = {'funder': funder}
    for i, p in tqdm(enumerate(iterate_publications_as_json(max_results=max_results, filter=filters))):
        pub = {}
        for f in p.get('funder'):
            if f.get('DOI') == '10.13039/501100002347':
                pub['AWARD'] = f.get('award','')
                break
        pub['DOI'] = p.get('DOI','')
        pub['TITLE'] = p.get('title','')
        pub['ABSTRACT'] = p.get('abstract','')
        if ('resource' in p.keys()) and ('primary' in p['resource'].keys()):
            pub['RESOURCE'] = p['resource']['primary'].get('URL','')
        else:
            pub['RESOURCE']=''

        metadatas.append(pub)
        
    df = pd.DataFrame.from_records(metadatas)
    df.to_csv('../data/pub_metadatas.csv')

    return df


In [14]:
# Crossref does not store the publications or papers but only the metadatas of these publications/ papers. Out of the metadatas, ABSTRACT is a brief summarization of the publication, which can be used for our enrichment purpose 
metadatas = retrieve_metadata(funder='501100002347', max_results = 500)
metadatas


500it [00:05, 98.26it/s]


,AWARD,DOI,TITLE,ABSTRACT,RESOURCE
0,[01IS14014A-D],10.1109/cist49399.2021.9357189,[Corpus and Baseline Model for Domain-Specific Entity Recognition in German],,https://ieeexplore.ieee.org/document/9357189/
1,[0339370],10.5194/soil-2016-71,[Process-oriented modelling to identify main drivers of erosion-induced carbon fluxes],"<jats:p>Abstract. Over the last few decades, modelling soil erosion and carbon redistribution has received great attention due to large uncertainties and conflicting results. For a process-oriented representation of event dynamics, coupled soil-carbon erosion models have been developed. However, there are currently few models that represent tillage erosion, preferential water erosion and transport of different carbon fractions (e.g. mineral bound carbon, carbon encapsulated by soil aggregates). We couple a process-oriented multi-class sediment transport model with a carbon turnover model (MCST-C) to identify relevant redistribution processes for carbon dynamics. The model is applied for two arable catchments (3.7 and 7.8 ha) located in the Tertiary hills about 40 km north of Munich, Germany. Our findings indicate the following: (i) Redistribution by tillage has a large effect on erosion-induced vertical carbon fluxes and has a large carbon sequestration potential. (ii) Water erosion has a minor effect on vertical fluxes, but episodic SOC delivery controls the long-term erosion-induced carbon balance. (iii) Delivered sediments are highly enriched in SOC compared to their parent material, and sediment delivery is driven by event size and catchment connectivity. (iv) Soil aggregation enhances SOC deposition due to the transformation of highly mobile carbon rich fine primary particles into rather immobile soil aggregates.\n </jats:p>",https://soil.copernicus.org/preprints/soil-2016-71/soil-2016-71.pdf
2,"[""Cultural Exchange from Syria to Ethiopia""]",10.1017/s0028688521000151,"[The Anonymous Source for Marcion's Gospel in British Library, Add. 17215: An Identification and Analysis]","<jats:p>For over a century, studies on Marcion have cited a quotation attributed to him in a fragmentary Syriac manuscript: London, British Library, Add. 17215 (fols. 30–3). An English translation of the relevant passage appeared in 1893, but no subsequent study has returned to the Syriac text itself. While this text has hitherto been cited as an anonymous Syriac source, this article identifies it as a letter by Jacob of Serugh (d. 520/1) and offers preliminary remarks on the implications of this identification for future research on Marcion's Gospel and his thought.</jats:p>",https://www.cambridge.org/core/product/identifier/S0028688521000151/type/journal_article
3,"[05K13WA1, 05K2013]",10.1103/physrevb.95.214512,"[Optical investigation of \n<mml:math xmlns:mml=""http://www.w3.org/1998/Math/MathML""><mml:mrow><mml:msub><mml:mi>BaFe</mml:mi><mml:mn>2</mml:mn></mml:msub><mml:msub><mml:mrow><mml:mo>(</mml:mo><mml:msub><mml:mi>As</mml:mi><mml:mrow><mml:mn>0.77</mml:mn></mml:mrow></mml:msub><mml:msub><mml:mi mathvariant=""normal"">P</mml:mi><mml:mrow><mml:mn>0.23</mml:mn></mml:mrow></mml:msub><mml:mo>)</mml:mo></mml:mrow><mml:mn>2</mml:mn></mml:msub></mml:mrow></mml:math>\n: Spin-fluctuation-mediated superconductivity under pressure]",,https://link.aps.org/doi/10.1103/PhysRevB.95.214512
4,,10.1186/isrctn08870215,[Internet-based relapse prevention for eating disorders following inpatient treatment: randomized controlled trial for bulimia nervosa],,http://www.isrctn.com/ISRCTN08870215
...,...,...,...,...,...
495,"[01GW0651-4, 027198]",10.1016/j.neuroimage.2013.01.002,[Neural correlates of spatial working memory load in a delayed match-to-sample saccade task],,https://linkinghub.elsevier.com/retrieve/pii/S1053811913000335
496,[15S9410B],10.1016/j.nimb.2022.09.027,[14C AMS measurement of gas diluted reactor graphite using a Gas Injection System],,https://linkinghub.elsevier.com/retrieve/pii/S0168583X22002701
497,,10.1016/j.gca.2012.03.0

In [25]:
metadatas =pd.read_csv("../data/crossref_pub_metadata.csv")

In [26]:
metadatas

,AWARD,DOI,TITLE,ABSTRACT,RESOURCE
0,['031B0733A'],10.3390/su13158651,['LED versus HPS Lighting: Effects on Water and Energy Consumption and Yield Quality in Lettuce Greenhouse Production'],"<jats:p>High-pressure sodium (HPS) lighting is increasingly replaced by LED lighting in lettuce greenhouse cultivation. In contrast to HPS lighting, LEDs do not heat radiation. Therefore, the leaf temperature is significantly lower under LEDs. This raises the question of whether LED lighting has a positive impact on the reduction in water consumption during lettuce production. In this paper, we investigated this question and found that the water consumption of lettuce produced under LEDs was significantly lower (−15%) than under HPS without loss of yield. We also found that supplementary lighting increases the concentrations of caffeoylquinic acid, dicaffeoyltartaric acid, dicaffeoylquinic acid and that of the total phenolic compounds in lettuce leaves by 61%, 39%, 163% and 38%, respectively. Only the LED fixture was also efficient enough to increase the concentration of caffeoyltartaric acid (+24%). Most of the phenolic compounds showed a very strong positive correlation with the chlorophyll concentration in lettuce, which predominated in the leaves exposed to the LED lighting. Based on these facts, we conclude that by optimizing the light composition, more sustainable plant production, higher concentrations of chlorophyll and some phenolic compounds are possible.</jats:p>",https://www.mdpi.com/2071-1050/13/15/8651
1,['01IS14014A-D'],10.1109/cist49399.2021.9357189,['Corpus and Baseline Model for Domain-Specific Entity Recognition in German'],NaN,https://ieeexplore.ieee.org/document/9357189/
2,['031B0653'],10.3390/bios12020062,['Electrophoretic µPAD for Purification and Analysis of DNA Samples'],"<jats:p>In this work, the fabrication and characterization of a simple, inexpensive, and effective microfluidic paper analytic device (µPAD) for monitoring DNA samples is reported. The glass microfiber-based chip has been fabricated by a new wax-based transfer-printing technique and an electrode printing process. It is capable of moving DNA effectively in a time-dependent fashion. The nucleic acid sample is not damaged by this process and is accumulated in front of the anode, but not directly on the electrode. Thus, further DNA processing is feasible. The system allows the DNA to be purified by separating it from other components in sample mixtures such as proteins. Furthermore, it is demonstrated that DNA can be moved through several layers of the glass fiber material. This proof of concept will provide the basis for the development of rapid test systems, e.g., for the detection of pathogens in water samples.</jats:p>",https://www.mdpi.com/2079-6374/12/2/62
3,NaN,10.1177/1535370218820290,['<i>In vitro</i> gene expression and detergent-free reconstitution of active proteorhodopsin in lipid vesicles'],"<jats:p> In situ generation of proton gradients across membranes is a key component for energy generation within cells and is therefore an important feature for the design of energy converting artificial cells. Here, we establish a stepwise method for the in vitro expression and detergent-free reconstitution of proteorhodopsin into the membrane of lipid vesicles. This represents a novel technique towards the bottom-up synthesis of energy-generating artificial cells. </jats:p><jats:sec><jats:title>Impact statement</jats:title><jats:p> Our results offer the potential for straightforward, additive-free, and molecularly simple routes to building complex bioreactors based on in vitro transcription–translation systems and lipid vesicles. </jats:p></jats:sec>",http://journals.sagepub.com/doi/10.1177/1535370218820290
4,['05K13PA1'],10.1039/c6nj02498j,"['Cr<sub>2</sub>(NCN)<sub>3</sub>, a ferromagnetic carbodiimide with an unusual two-step magnetic transition']",<p>Crystallographic and magnetic characterization of a ferromagnetic transition-metal carbodiimide

In [27]:
#remove html tag
metadatas.loc[~metadatas["ABSTRACT"].isnull(),"ABSTRACT"] = metadatas.loc[~metadatas["ABSTRACT"].isnull(),"ABSTRACT"].apply(remove_html_tags)


In [28]:
# determine the language of the publication 
metadatas.loc[~metadatas["ABSTRACT"].isnull(),"lang"] = metadatas.loc[~metadatas["ABSTRACT"].isnull(),"ABSTRACT"].apply(detect_language)
   

In [29]:
# from all the abstracts only 138 abstracts are in the German language
metadatas[metadatas["lang"]=="de"].shape

(138, 6)

In [30]:
metadatas[metadatas["lang"]=="de"]

,AWARD,DOI,TITLE,ABSTRACT,RESOURCE,lang
271,['01JD1805'],10.1007/s40955-021-00185-4,['Digitalisierung in Angeboten der Erwachsenenbildung/Weiterbildung'],"ZusammenfassungDigitalisierung und Mediatisierung prägen die Gesellschaft und auch die Erwachsenenbildung/Weiterbildung. Der Beitrag geht der Frage nach, wie Digitalisierung in Angeboten der Erwachsenenbildung/Weiterbildung gelingt. Damit wird ein Fokus auf den Einsatz digitaler Medien gelegt. Dazu werden die Angebotsentwicklung für Adressatinnen und Adressaten sowie Teilnehmende, medienbezogene Inhalte, Lehr- und Lernarrangements mit digitalen Medien, der Einsatz digitaler Medien und die Zugänglichkeit von Lehr- und Lernmaterialien als relevante Merkmale identifiziert. Insgesamt zeigen die analysierten Interviewdaten, dass der Einsatz digitaler Medien in Angeboten eine Erweiterung der didaktischen Aufgaben darstellt, da Angebote mit digitalen Medien zielgenau auf die Bedarfe und Möglichkeiten von Adressatinnen und Adressaten sowie Teilnehmenden abgestimmt werden müssen.",https://link.springer.com/10.1007/s40955-021-00185-4,de
354,['01JA1621'],10.1007/s42010-022-00149-8,['Selbstvideographie im Vorbereitungsdienst: Eine Implementationsstudie zur Förderung der klassenführungsbezogenen Unterrichtswahrnehmung von Lehramtsanwärter*innen'],"ZusammenfassungKlassenführung ist eine zentrale Komponente effektiven Unterrichtens. Jedoch haben Lehrkräfte vor allem zu Beginn der berufspraktischen Tätigkeit häufig Schwierigkeiten, mit klassenführungsbezogenen Herausforderungen umzugehen. Dies führt nicht selten zu Burnout oder beruflichem Dropout. Um solchen Entwicklungen entgegenzusteuern, sollten Lehrkräfte hinsichtlich ihres Unterrichts so früh wie möglich eine klassenführungssensible professionelle Wahrnehmung entwickeln. Dazu wurde eine entsprechende universitäre Qualifizierungsmaßnahme mit Selbstvideographieelementen anhand von 121 Lehramtsanwärter*innen in den Vorbereitungsdienst implementiert. Ziel der Studie war es herauszufinden, inwiefern und unter welchen Bedingungen sich die Maßnahme am besten auf die Entwicklung der professionellen Wahrnehmung, erhoben mit Hilfe eines validierten videobasierten Tests, auswirkt. Bei der Förderung der professionellen Wahrnehmung von Klassenführung konnte sich die Methode der Selbstvideographie inklusive anschließender Videoanalyse vor dem Hintergrund des hochsignifikanten Haupteffekts des Vorbereitungsdiensts nicht gegenüber dem Effekt von traditionellen Unterrichtsbesuchen durchsetzen. Ein ähnlicher Effekt zeigte sich bei dem Vergleich von Peer- und Expertenfeedback innerhalb der Selbstvideographiegruppe. Interessanterweise waren jedoch systematische Unterschiede in der Veränderung der professionellen Wahrnehmung auf Grundlage der standortspezifischen Implementationsbedingungen erkennbar. Die Bedeutung der Ergebnisse für die Übertragung von Innovationen in den Vorbereitungsdienst wird diskutiert.",https://link.springer.com/10.1007/s42010-022-00149-8,de
519,['01LZ1703D'],10.1007/s00548-021-00708-8,['Anpassung an Überschwemmungen in südostasiatischen Megastädten\xa0– Was kann der Privatsektor leisten?'],"ZusammenfassungDie fortschreitende Urbanisierung und erste Auswirkungen des Klimawandels verstärken das Überschwemmungsrisiko in küstennahen südostasiatischen Megastädten. Lokale Industriebetriebe sind insbesondere in ihrer Funktions- und Wettbewerbsfähigkeit betroffen. Gleichzeitig wird in den Debatten zu Klimawandelanpassung und Katastrophenrisikominderung die Beteiligung des Privatsektors an Flutschutzmaßnahmen diskutiert, da die lokalen Behörden in Südostasien das Flutrisiko häufig nicht eigenständig substanziell reduzieren können. Vor diesem Hintergrund wird untersucht, wie Industriebetriebe in Ho Chi Minh City (Vietnam) und Jakarta (Indonesien) auf Überschwemmungen reagieren, welche Anpassungsmaßnahmen sie durchführen und inwiefern ihr Engagement einen Beitrag zur kollektiven Anpassungen leistet. Die Analyse von 63 Tiefeninterviews mit Ind

In [31]:
# 19 of 138 publication in german do not have funder number 
metadatas.loc[(metadatas["lang"]=="de") & (metadatas["AWARD"].isnull())]

,AWARD,DOI,TITLE,ABSTRACT,RESOURCE,lang
1219,NaN,10.1007/s11618-021-01023-6,['Außerunterrichtliches Peer Tutoring mit deutsch-türkischsprachigen Grundschulkindern: Umsetzungsgenauigkeit und Umfang von bilingualer Kommunikation'],"ZusammenfassungDer vorliegende Beitrag fokussiert die Implementation von außerunterrichtlichen Peer Tutoring-Trainings im Lesen und Rechnen mit türkisch-deutschsprachigen Grundschulkindern. Hierbei bildeten jeweils zwei Grundschulkinder (Peers) ein Tandem, das von einer geschulten studentischen Trainingsleitung angeleitet wurde. Zur Erfassung der Implementation von Peer Tutoring-Elementen unter diesen Bedingungen wurde die Umsetzungsgenauigkeit herangezogen. Eine Besonderheit der Studie war, dass es einem Teil der Trainingsgruppen erlaubt war, beide Sprachen während des Trainings zu sprechen. Für die bilinguale Kommunikation wurden zusätzliche Operationalisierungen geprüft, um die Umsetzung bilingualer Kommunikation zu erfassen. Darüber hinaus wird im Beitrag konsequent zwischen der Implementation durch Trainingsleitungen und durch Lernende unterschieden. Die Ergebnisse unterstreichen die hohe Umsetzungsgenauigkeit der außerunterrichtlichen Peer Tutoring-Trainings. Sie zeigen aber auch auf, dass die Lernenden in diesem Setting nur wenig miteinander in ihrer Herkunftssprache kommunizierten und mehr türkischsprachige Impulse durch Trainingsleitungen nicht mit mehr bilingualer Kommunikation durch die Lernenden einherging. Der Beitrag liefert somit weitere Erkenntnisse zur Aktivierung und zum Umfang bilingualer Kommunikation beim kooperativen Lernen.",https://link.springer.com/10.1007/s11618-021-01023-6,de
1239,NaN,10.1365/s40702-021-00798-w,['Unterstützung digitaler Bildungsprozesse durch interaktive gamifizierte Lernvideos\xa0– Wie innovative Lernvideos Motivation und Lernerfolg steigern können'],"ZusammenfassungDigitales Lernen ist ein zentraler Bestandteil der heutigen Bildungslandschaft geworden. Insbesondere Lernvideos haben in den letzten Jahren deutlich an Popularität gewonnen und Plattformen wie YouTube sind heute ein fester Bestandteil von Lernprozessen. Bildungseinrichtungen haben diese Entwicklung erkannt und Lernvideos in ihre Lernsysteme integriert. Allerdings sind die heutigen Lernvideos oft nicht sehr ansprechend, weil es an Gestaltungswissen mangelt, wie Interaktionen und gamifizierte Elemente die Motivation und den Lernerfolg steigern können. Dieser Mangel an Gestaltungswissen führt zu minderwertigen Lernvideos, die den Lernprozess der Lernenden meist nicht in dem Maße unterstützen, wie sie es könnten. Um effektive Lernvideos zu entwickeln, welche die Lernenden adäquat unterstützen, ist handfestes und praktisches Gestaltungswissen für die Lehrenden notwendig. Vor diesem Hintergrund entwickeln wir Entwurfsmuster für interaktive gamifizierte Lernvideos, um Praktiker*innen bei der Erstellung effektiver Lernvideos anzuleiten, die die Motivation und das Engagement von Lernenden zu fördern, welche sie aktiv in ihrem Lernprozess unterstützen zu können. Wir setzen damit zusätzlich die Grundlage für zukünftige Forschung zur Gestaltung effektiver Lernvideos. Dabei folgen wir einem gestaltungsorientierten Forschungsansatz, um die Entwicklung unserer Entwurfsmuster zu systematisieren.",https://link.springer.com/10.1365/s40702-021-00798-w,de
6118,NaN,10.1007/s40955-022-00206-w,['Teilnehmendengewinnung in der Weiterbildung und spezifische Ansprachestrategien in der Alphabetisierung und Grundbildung\xa0– ein systematisches Literaturreview'],"ZusammenfassungTeilnehmende für Weiterbildungsangebote zu gewinnen, stellt eine zentrale Herausforderung der Weiterbildungspraxis dar und ist eine klassische Aufgabe des Weiterbildungspersonals. Innerhalb des Weiterbildungsdiskurses gibt es für diesen Themenbereich jedoch keinen einheitlichen Terminus bzw. keine etablierte Kategorie. Der Beitrag zielt daher auf die Bestandsaufnahme und Systematisierung des Diskussions- und Forschungsstands zum Gegenstandsbereich de

In [32]:
# remove character [] and '' in column AWARD
metadatas.loc[(metadatas["lang"]=="de") & (~metadatas["AWARD"].isnull()),"AWARD"]= metadatas.loc[(metadatas["lang"]=="de") & (~metadatas["AWARD"].isnull()),"AWARD"].apply(lambda li: li.split("'")[1])

In [33]:
de_metadatas = metadatas.loc[(metadatas["lang"]=="de") & (~metadatas["AWARD"].isnull())]

In [34]:
de_metadatas = split_entries_in_column(de_metadatas, "AWARD")
de_metadatas

,AWARD,DOI,TITLE,ABSTRACT,RESOURCE,lang
271,01JD1805,10.1007/s40955-021-00185-4,['Digitalisierung in Angeboten der Erwachsenenbildung/Weiterbildung'],"ZusammenfassungDigitalisierung und Mediatisierung prägen die Gesellschaft und auch die Erwachsenenbildung/Weiterbildung. Der Beitrag geht der Frage nach, wie Digitalisierung in Angeboten der Erwachsenenbildung/Weiterbildung gelingt. Damit wird ein Fokus auf den Einsatz digitaler Medien gelegt. Dazu werden die Angebotsentwicklung für Adressatinnen und Adressaten sowie Teilnehmende, medienbezogene Inhalte, Lehr- und Lernarrangements mit digitalen Medien, der Einsatz digitaler Medien und die Zugänglichkeit von Lehr- und Lernmaterialien als relevante Merkmale identifiziert. Insgesamt zeigen die analysierten Interviewdaten, dass der Einsatz digitaler Medien in Angeboten eine Erweiterung der didaktischen Aufgaben darstellt, da Angebote mit digitalen Medien zielgenau auf die Bedarfe und Möglichkeiten von Adressatinnen und Adressaten sowie Teilnehmenden abgestimmt werden müssen.",https://link.springer.com/10.1007/s40955-021-00185-4,de
354,01JA1621,10.1007/s42010-022-00149-8,['Selbstvideographie im Vorbereitungsdienst: Eine Implementationsstudie zur Förderung der klassenführungsbezogenen Unterrichtswahrnehmung von Lehramtsanwärter*innen'],"ZusammenfassungKlassenführung ist eine zentrale Komponente effektiven Unterrichtens. Jedoch haben Lehrkräfte vor allem zu Beginn der berufspraktischen Tätigkeit häufig Schwierigkeiten, mit klassenführungsbezogenen Herausforderungen umzugehen. Dies führt nicht selten zu Burnout oder beruflichem Dropout. Um solchen Entwicklungen entgegenzusteuern, sollten Lehrkräfte hinsichtlich ihres Unterrichts so früh wie möglich eine klassenführungssensible professionelle Wahrnehmung entwickeln. Dazu wurde eine entsprechende universitäre Qualifizierungsmaßnahme mit Selbstvideographieelementen anhand von 121 Lehramtsanwärter*innen in den Vorbereitungsdienst implementiert. Ziel der Studie war es herauszufinden, inwiefern und unter welchen Bedingungen sich die Maßnahme am besten auf die Entwicklung der professionellen Wahrnehmung, erhoben mit Hilfe eines validierten videobasierten Tests, auswirkt. Bei der Förderung der professionellen Wahrnehmung von Klassenführung konnte sich die Methode der Selbstvideographie inklusive anschließender Videoanalyse vor dem Hintergrund des hochsignifikanten Haupteffekts des Vorbereitungsdiensts nicht gegenüber dem Effekt von traditionellen Unterrichtsbesuchen durchsetzen. Ein ähnlicher Effekt zeigte sich bei dem Vergleich von Peer- und Expertenfeedback innerhalb der Selbstvideographiegruppe. Interessanterweise waren jedoch systematische Unterschiede in der Veränderung der professionellen Wahrnehmung auf Grundlage der standortspezifischen Implementationsbedingungen erkennbar. Die Bedeutung der Ergebnisse für die Übertragung von Innovationen in den Vorbereitungsdienst wird diskutiert.",https://link.springer.com/10.1007/s42010-022-00149-8,de
519,01LZ1703D,10.1007/s00548-021-00708-8,['Anpassung an Überschwemmungen in südostasiatischen Megastädten\xa0– Was kann der Privatsektor leisten?'],"ZusammenfassungDie fortschreitende Urbanisierung und erste Auswirkungen des Klimawandels verstärken das Überschwemmungsrisiko in küstennahen südostasiatischen Megastädten. Lokale Industriebetriebe sind insbesondere in ihrer Funktions- und Wettbewerbsfähigkeit betroffen. Gleichzeitig wird in den Debatten zu Klimawandelanpassung und Katastrophenrisikominderung die Beteiligung des Privatsektors an Flutschutzmaßnahmen diskutiert, da die lokalen Behörden in Südostasien das Flutrisiko häufig nicht eigenständig substanziell reduzieren können. Vor diesem Hintergrund wird untersucht, wie Industriebetriebe in Ho Chi Minh City (Vietnam) und Jakarta (Indonesien) auf Überschwemmungen reagieren, welche Anpassungsmaßnahmen sie durchführen und inwiefern ihr Engagement einen Beitrag zur kollektiven Anpassungen leistet. Die Analyse von 63 Tiefeninterviews mit Industriebetrie

In [37]:
de_metadatas["AWARD"] = de_metadatas["AWARD"].apply(extract_FKZ)


In [56]:
#remove character ' in column TITLE  
de_metadatas.loc[:,"TITLE"] = de_metadatas.loc[:,"TITLE"].apply(lambda string:string.split("'")[1])

271                                                                                                       Digitalisierung in Angeboten der Erwachsenenbildung/Weiterbildung
354      Selbstvideographie im Vorbereitungsdienst: Eine Implementationsstudie zur Förderung der klassenführungsbezogenen Unterrichtswahrnehmung von Lehramtsanwärter*innen
519                                                                  Anpassung an Überschwemmungen in südostasiatischen Megastädten\xa0– Was kann der Privatsektor leisten?
546                                                               Ein Instrument zur Messung von Lerntransferbedingungen im Kontext der hochschuldidaktischen Weiterbildung
546                                                               Ein Instrument zur Messung von Lerntransferbedingungen im Kontext der hochschuldidaktischen Weiterbildung
                                                                                        ...                                                 

In [57]:
BMBF = pd.read_csv("../data/cleanedBMBF.csv")
BMBF.head(2)

,FKZ,Ressort,Referat,PT,Arb.-Einh.,Zuwendungsempfänger,Gemeindekennziffer,Stadt/Gemeinde,Ort,Bundesland,Staat,Ausführende Stelle,Gemeindekennziffer.1,Stadt/Gemeinde.1,Ort.1,Bundesland.1,Staat.1,Thema,Leistungsplansystematik,Klartext Leistungsplansystematik,Laufzeit von,Laufzeit bis,Fördersumme in EUR,Förderprofil,Verbundprojekt,Förderart
0,03F0212C/7,BMBF,724,PT-J,MGS1,Universität Rostock,13003000.0,Rostock,Rostock,Mecklenburg-Vorpommern,Deutschland,Universität Rostock - Fachbereich Elektrotechnik und Informationstechnik,13003000.0,Rostock,Rostock,Mecklenburg-Vorpommern,Deutschland,"Verbundprojekt MESSIN: Entwicklung und Systemintegration der Komponenten Navigation, Automatische Steuerung, Kommunikation und Energieversorgung für den Meßdelphin MESSIN",FB4010,"Technik-/Infrastrukturentwicklung und -bereitstellung (Überwachungssysteme, Begleitforschung zum Monitoring, innovative Technik und Geräteträger)",01.01.1998,31.08.2000,"648.462,00",Forschung und Entwicklung zur Daseinsvorsorge,MESSIN,PDIR
1,D251800/0,BMBF,314,BIBB,A3.3,"Ministerium für Wirtschaft, Arbeit und Wohnungsbau Baden-Württemberg",8111000.0,Stuttgart,Stuttgart,Baden-Württemberg,Deutschland,KACO new energy GmbH,8121000.0,Heilbronn,Heilbronn,Baden-Württemberg,Deutschland,Prozeßorientierte Entwicklungsplanung und Qualifizierung im Betrieb,OB1050,Versuchs- und Modelleinrichtungen und -programme im Bereich der beruflichen Bildung,01.07.1997,30.06.2000,"96.634,00",Forschung und Entwicklung zur Daseinsvorsorge,NaN,PDIR


In [65]:
# there are 68 
print(len(set(BMBF["FKZ"]).intersection(set(de_metadatas["AWARD"]))))

In [66]:
list(set(BMBF["FKZ"]).intersection(set(de_metadatas["AWARD"])))

In [67]:
de_metadatas.set_index("AWARD",inplace=True)
BMBF.set_index("FKZ",inplace=True)
BMBF_crossref_enrichment = BMBF.join(de_metadatas,how="outer")
BMBF_crossref_enrichment

In [68]:
BMBF_crossref_enrichment.loc[list(set(BMBF.index).intersection(set(de_metadatas.index))),:]

In [69]:
BMBF_crossref_enrichment.loc[list(set(BMBF.index).intersection(set(de_metadatas.index))),"TITLE"].iloc[2]

In [109]:
def split_entries_in_column(df, column_name):
    # Duplicate rows with multiple values in the specified column
    df_split = df.copy()
    df_split[column_name] = df_split[column_name].str.split('[&|;]')

    # Explode the lists in the specified column to create multiple rows
    df_split = df_split.explode(column_name)

    return df_split

